## Transfer Learning VGG 16 and VGG 19 using Keras

Please download the dataset from the below url

In [1]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 
    

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob


In [4]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = r'C:\Users\50021726\Desktop\Ineuron\catdogdata\training_set'
valid_path = r'C:\Users\50021726\Desktop\Ineuron\catdogdata\test_set'


In [5]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




In [6]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [7]:
  # useful for getting number of output classes
folders = glob(r'C:\Users\50021726\Desktop\Ineuron\catdogdata\training_set/*')

In [8]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

In [9]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [10]:

# view the structure of the model
model.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [11]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [12]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(r'C:\Users\50021726\Desktop\Ineuron\catdogdata\training_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 8005 images belonging to 2 classes.


In [14]:
test_set = test_datagen.flow_from_directory(r'C:\Users\50021726\Desktop\Ineuron\catdogdata\test_set',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                          class_mode = 'categorical')

Found 2023 images belonging to 2 classes.


In [15]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [16]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
251/251 [==============================] - 81s 324ms/step - loss: 0.3242 - accuracy: 0.8637 - val_loss: 0.4202 - val_accuracy: 0.8393
Epoch 2/10
251/251 [==============================] - 84s 333ms/step - loss: 0.2114 - accuracy: 0.9127 - val_loss: 0.1796 - val_accuracy: 0.9283
Epoch 3/10
251/251 [==============================] - 80s 320ms/step - loss: 0.1913 - accuracy: 0.9237 - val_loss: 0.4161 - val_accuracy: 0.8606
Epoch 4/10
251/251 [==============================] - 80s 318ms/step - loss: 0.1597 - accuracy: 0.9342 - val_loss: 0.1969 - val_accuracy: 0.9323
Epoch 5/10
251/251 [==============================] - 81s 321ms/step - loss: 0.1463 - accuracy: 0.9438 - val_loss: 0.2205 - val_accuracy: 0.9224
Epoch 6/10
251/251 [==============================] - 84s 333ms/step - loss: 0.1409 - accuracy: 0.9455 - val_loss: 0.1888 - val_accuracy: 0.9397
Epoch 7/10
251/251 [==============================] - 

In [ ]:
# plot the loss
%matplotlib
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
pip install matplotlib

In [ ]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('model_vgg16.h5')

In [17]:

y_pred = model.predict(test_set)


In [18]:
y_pred

array([[1.8584134e-02, 9.8141587e-01],
       [7.7653760e-01, 2.2346240e-01],
       [8.4470754e-04, 9.9915528e-01],
       ...,
       [1.1822557e-05, 9.9998820e-01],
       [9.8070908e-01, 1.9290902e-02],
       [9.4475734e-01, 5.5242646e-02]], dtype=float32)

In [19]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [20]:
y_pred

array([1, 0, 1, ..., 1, 0, 0], dtype=int64)

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('model_resnet50.h5')

In [ ]:
img_data

In [22]:
img=image.load_img(r'C:\Users\50021726\Desktop\Ineuron\catdogdata\test_set\dogs\dog.4011.jpg',target_size=(224,224))



In [23]:
x=image.img_to_array(img)
x

array([[[ 54.,  69.,  28.],
        [108., 126.,  86.],
        [ 90., 111.,  72.],
        ...,
        [ 25.,  50.,  29.],
        [ 29.,  56.,  41.],
        [ 31.,  57.,  44.]],

       [[ 44.,  61.,  19.],
        [ 45.,  61.,  22.],
        [ 39.,  55.,  16.],
        ...,
        [ 21.,  48.,  31.],
        [ 24.,  55.,  37.],
        [ 21.,  52.,  34.]],

       [[ 50.,  68.,  26.],
        [ 53.,  68.,  29.],
        [ 52.,  62.,  25.],
        ...,
        [ 26.,  53.,  38.],
        [ 23.,  55.,  34.],
        [ 43.,  76.,  55.]],

       ...,

       [[245., 246., 241.],
        [246., 247., 242.],
        [247., 248., 243.],
        ...,
        [194., 193., 199.],
        [191., 191., 193.],
        [199., 199., 199.]],

       [[247., 248., 243.],
        [248., 249., 244.],
        [248., 249., 244.],
        ...,
        [194., 193., 199.],
        [190., 190., 192.],
        [197., 197., 197.]],

       [[250., 251., 246.],
        [250., 251., 246.],
        [249., 2

In [ ]:
x.shape

In [24]:
x=x/255

In [25]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [26]:
model.predict(img_data)

array([[0.2939851 , 0.70601493]], dtype=float32)

In [27]:
a=np.argmax(model.predict(img_data), axis=1)

In [29]:
a

array([1], dtype=int64)

In [30]:
if(a==0):
    print("The image classified is cat")
else:
    print("The image classified is dog")

The image classified is dog
